# Rayyan SDK for Python

## Installation

After installing the package you can import it and create a `rayyan` instance as follows:

> Note: the python package is not yet available on PyPI, so you need to install it from the source code.

In [1]:
from pprint import pprint
from rayyan import Rayyan

rayyan = Rayyan("creds.json", "https://staging.rayyan.ai")

The `creds.json` file should contain the following:

```json
{
    "access_token": "...",
    "refresh_token": "..."
}
```

A valid file can be downloaded by signing in to Rayyan then going to My Account.

> Note: this is not supported yet in production for all users.

## Accessing signed-in user data

The simplest SDK call is the one that retrieves the signed-in user's data. This is done by calling the `get_info` method of `User` object:

In [2]:
from rayyan.user import User

user = User(rayyan).get_info()

# Remove request_token before printing
user.pop('request_token', None)
# Remove some other fields
user.pop('subscription', None)
# user.pop('feature_flags', None)
pprint(user)

{'displayName': 'MR test test2',
 'feature_flags': {'add_review_metadata_to_review_form': True,
                   'auto_resolver': True,
                   'customize_by_filter': True,
                   'feos_access': True,
                   'feos_banner': True,
                   'full_text': False,
                   'live_chat': False,
                   'mendeley_import': True,
                   'mobile_access': False,
                   'phone_support': False,
                   'pico_extraction': True,
                   'prisma': True,
                   'revoked_roles': True,
                   'training': False},
 'firstName': 'test',
 'lastName': 'test2',
 'publicName': 'test test2',
 'userId': 266,
 'user_email': 'mohammad+team+owner@rayyan.ai'}


## Retrieving the list of all reviews

In [3]:
from rayyan.review import Review
rayyan_review = Review(rayyan)
reviews = rayyan_review.get_all()
pprint(reviews)
print(f'Found {len(reviews["owned_reviews"])} owned reviews and {len(reviews["collab_reviews"])} shared reviews.')

{'collab_reviews': [],
 'collab_reviews_total': 0,
 'owned_reviews': [{'accessed_at': '2023-09-07T11:09:39.141Z',
                    'articles_etag': '3408:10-3409:5794',
                    'created_at': '2023-09-06T13:08:40.060Z',
                    'inclusions_etag': '2023-09-07 10:32:21 UTC',
                    'is_blind': True,
                    'labels_etag': '',
                    'maybe_decision_supported': True,
                    'owner': {'abbrev': 'test',
                              'email': 'mohammad+team+owner@rayyan.ai',
                              'enterprise_power_user?': False,
                              'global_owner_user?': False,
                              'guest?': False,
                              'has_fulltext_access': True,
                              'id': 266,
                              'most_recent_active_single_team_subscription': None,
                              'most_recent_active_team_subscription': {'cancelled_at': None,
    

## Working on the first owned review

### Printing some review key information

In [4]:
my_review = reviews['owned_reviews'][0]
# pprint(my_review)
print(f"Here is the first review in the list of owned reviews: '{my_review['title']}' created on '{my_review['created_at']}' and it has a total of {my_review['total_articles']} articles.")


Here is the first review in the list of owned reviews: 'Review test py client' created on '2023-09-06T13:08:40.060Z' and it has a total of 5804 articles.


### Listing the first 15 articles sorted by title

In [5]:
review_id = my_review['rayyan_id']
result_params = {
    "start": 0,
    "length": 15,
    "order[0][column]": 5,
    "order[0][dir]": "asc"
    # ... other query parameters ...
}
review_results = rayyan_review.results(review_id, result_params)
print(f'Returned {len(review_results["data"])} record(s) matching {review_results["recordsFiltered"]} record(s) out of {review_results["recordsTotal"]} in total')


Returned 15 record(s) matching 5804 record(s) out of 5804 in total


### Printing article titles and authors

In [6]:
for index, result in enumerate(review_results['data']):
    print(f'{index+1}: {result["title"]}')
    print(f'    Authors: {", ".join(result["authors"])}')

1: 10-Year Comparative Follow-up of Familial versus Multifactorial Chylomicronemia Syndromes.
    Authors: Belhassen M, Van Ganse E, Nolin M, Bérard M, Bada H, Bruckert E, Krempf M, Rebours V, Valero R, Moulin P
2: 11C-L-methionine for evaluation of pancreatic exocrine function.
    Authors: Syrota A, Dop-Ngassa M, Cerf M, Paraf A
3: [11C]methionine positron emission tomography for the evaluation of pancreatic exocrine function in chronic pancreatitis.
    Authors: Takasu A, Shimosegawa T, Shimosegawa E, Hatazawa J, Nagasaki Y, Kimura K, Fujita M, Toyota T
4: 13C-mixed triglyceride breath test for evaluation of pancreatic exocrine function in diabetes mellitus.
    Authors: Keller J, Layer P, Brückel S, Jahr C, Rosien U
5: [13C-mixed triglyceride CO2 exhalation test. Investigation with an isotope selective, non dispersive infrared spectrophotometer of indirect function of the exocrine pancreas].
    Authors: Adamek RJ, Bödeker C, Szymanski C, Hagemann D, Pfaffenbach B
6: 13C-starch bre

### Including the first article

In [7]:
my_article = review_results['data'][0]
article_id = my_article['id']
customization_plan = {
    "included": 1
}
rayyan_review.customize(review_id, article_id, customization_plan)
print(f'Included article with title: {my_article["title"]}')

Included article with title: 10-Year Comparative Follow-up of Familial versus Multifactorial Chylomicronemia Syndromes.


### Filtering by included articles

In [8]:
result_params = {
    "start": 0,
    "length": 20,
    "extra[mode]": "included"
}
included_results = rayyan_review.results(review_id, result_params)
print(f'Returned {len(included_results["data"])} record(s) matching {included_results["recordsFiltered"]} record(s) out of {included_results["recordsTotal"]} in total:')
for index, result in enumerate(included_results['data']):
    print(f'{index+1}: {result["title"]}')
    print(f'    Authors: {", ".join(result["authors"])}')


Returned 1 record(s) matching 1 record(s) out of 5804 in total:
1: 10-Year Comparative Follow-up of Familial versus Multifactorial Chylomicronemia Syndromes.
    Authors: Belhassen M, Van Ganse E, Nolin M, Bérard M, Bada H, Bruckert E, Krempf M, Rebours V, Valero R, Moulin P


### Excluding the next 5 articles

In [9]:
my_articles = review_results['data'][1:6]
customization_plan = {
    "included": -1
}
for article in my_articles:
    article_id = article['id']
    print(f'Excluding article {article_id}...')
    rayyan_review.customize(review_id, article_id, customization_plan)
print(f'Excluded {len(my_articles)} articles from review {review_id}.')

Excluding article 776311...
Excluding article 774908...
Excluding article 773192...
Excluding article 775073...
Excluding article 775228...
Excluded 5 articles from review 2755.


### Excluding the next 5 articles in bluk

In [10]:
my_articles = review_results['data'][7:10]
rayyan_review.bulk_customizations(review_id, "included", -1, (",").join(map(str,[a['id'] for a in my_articles])))

{'missed_etags': {'inclusions_etag': '2023-09-07 11:16:41 UTC'},
 'new_etags': {'inclusions_etag': '2023-09-07 11:16:41 UTC',
  'labels_etag': ''},
 'customization_ids': [306, 307, 308]}

### Getting the inclusion counts facet for the review

In [11]:
facet_params = {
    "facets[inclusion_counts]": "1"
}
facet_info = rayyan_review.facets(review_id, facet_params)
pprint(facet_info)

{'inclusion_counts': {'collection': [], 'has_evenmore': False, 'more': []}}


### Get `inclusion_decisions`, and `labels` Customizations for a first 15 articles sorted by title

In [12]:
customization_params = {
    "types[]": ["inclusion_decisions", "labels"]
}

pprint(rayyan_review.get_customizations(review_id, customization_params))

[{'_id': 265,
  'article_id': 776311,
  'created_at': '2023-09-06T13:14:04.274Z',
  'key': 'included',
  'user_id': 266,
  'value': '0'},
 {'_id': 266,
  'article_id': 776311,
  'created_at': '2023-09-06T13:15:27.271Z',
  'key': 'included',
  'user_id': 266,
  'value': '1'},
 {'_id': 267,
  'article_id': 776311,
  'created_at': '2023-09-06T13:15:55.944Z',
  'key': 'included',
  'user_id': 266,
  'value': '-1'},
 {'_id': 268,
  'article_id': 774908,
  'created_at': '2023-09-06T13:15:56.046Z',
  'key': 'included',
  'user_id': 266,
  'value': '-1'},
 {'_id': 269,
  'article_id': 773192,
  'created_at': '2023-09-06T13:15:56.196Z',
  'key': 'included',
  'user_id': 266,
  'value': '-1'}]


### Invite Collaborator to the review

In [14]:
from rayyan.review_access import ReviewAccess
review_access_instance = ReviewAccess(rayyan)

invite_confirmation = review_access_instance.invite(review_id, 2, ["user3@example.com"], "Collaboration on data analysis.")
pprint(invite_confirmation)


''


### Change collaborator to a viewer

In [15]:
invite_confirmation = review_access_instance.update_access(review_id, 3, ["user3@example.com"])
pprint(invite_confirmation)


''


### Revoke viewer access

In [16]:
# invite_confirmation = review_access_instance.update_access(review_id, 3, ["user3@example.com"])
# pprint(invite_confirmation)
my_review_json = rayyan_review.get(review_id)
viewr_id = my_review_json['viewers'][0]['id']

pprint(review_access_instance.revoke(review_id, viewr_id))


''


### Restore viewer access

In [17]:
pprint(review_access_instance.restore(review_id, viewr_id))
pprint(viewr_id)

''
594


### Delete viewer access

In [18]:
pprint(review_access_instance.delete_access(review_id, viewr_id))

''


## Notes

### Create note

In [36]:
from rayyan.notes import Notes

notes_instance = Notes(rayyan)
note_id = notes_instance.create_note(review_id, my_articles[0]['id'], text="This article needs further analysis.")
print(f'Added a note to article: {my_articles[0]["title"]} \nNote id: {note_id}')


Added a note to article: 18FDG-positron emission tomography in pancreatic cancer. 
Note id: 2529


### Update Note

In [37]:
response = notes_instance.update_note(review_id, my_articles[0]['id'], note_id, text="UPDATED This article needs further analysis.")
print(response)


{'status': 'OK'}


### Delete Note

In [39]:
response = notes_instance.delete_note(review_id, my_articles[0]['id'], note_id)
print(response)


HTTPError: 404 Client Error: Not Found for url: https://staging.rayyan.ai/api/v1/reviews/2755/articles/775039/notes/2529

## ThirdPartyAuth

### Get ThirdPartyAuth link

In [22]:
from rayyan.third_party_auth import ThirdPartyAuth
from rayyan.conf import MENDELEY

third_party_auth_instance = ThirdPartyAuth(rayyan, MENDELEY)
auth_link = third_party_auth_instance.get_auth_link()
pprint(auth_link)

{'code': 302,
 'data': {'redirect_to': 'https://api.mendeley.com/oauth/authorize?client_id=14062&redirect_uri=https%3A%2F%2Fstaging.rayyan.ai%2Fthird_party_auth%2Fmendeley_callback&response_type=code&scope=all'},
 'message': 'Found'}


## Highlights

### Create Highlight

In [41]:
from rayyan.highlight import Highlight

highlight_instance = Highlight(rayyan)
highlight_data = highlight_instance.create_highlight(review_id, category_id=1, keyword="important term")
print(highlight_data)


OK


## Duplicates
### Trigger dedup job for the review

In [46]:
from rayyan.duplicates import Duplicates

duplicates_instance = Duplicates(rayyan)

duplicates_instance.add_duplicate(review_id)


{'triggered': True, 'reason': None}

In [47]:
duplicates_instance.get_duplicate(review_id,my_articles[0]['id'])


[]